In [0]:
import gym
import numpy as np
import random

from IPython.display import clear_output
from time import sleep

In [0]:
env = gym.make("Taxi-v3")
action = env.reset()

In [3]:
print(env.action_space.n)
print(env.observation_space.n)

6
500


In [4]:
# 환경 동작 확인
env.render()
state_t, reward, done, e = env.step(env.action_space.sample())
print(reward)
if done:
    env.reset()
    print(reward)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

-10


In [5]:
qtable = np.zeros((500, 6))
print(qtable)

[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]


In [0]:
# 파라메터 설정
total_episodes = 100000 # 총 에피소드 수
total_test_episodes = 100 # 총 테스트 에피소드
max_steps = 100 # 하나의 에피소드에서 최대로 움직일 수 있는 수

learning_rate = 0.3 # 학습률
gamma = 0.8 # 미래 지향적일지 아닐지 선택, 크면 클수록 근시안적임

#탐색 파라메터
epsilon = 1.0 # 탐색률
max_epsilon = 1.0 # 탐색 나갈 최대의 확률
min_epsilon = 0.01 # 탐색 나갈 최소의 확률
decay_rate = 0.0001 # 점점 탐색 확률을 줄여나갈 크기

rewards = 0

In [0]:
for episode in range(total_episodes):
    # 환경을 초기화 한다
    state = env.reset()
    step = 0
    done = False # 처음이니까 끝나는 것은 아님
    total_rewards = 0
    
    # max_step까지 계속 반복
    for step in range(max_steps):
        
        #행동을 선택한다.
        # 랜덤으로 모험을 하기위한 확률을 설정
        exploration_probability = random.uniform(0, 1)
        
        # 호기심 보단 지도를 보면서 다닌다.
        if exploration_probability > epsilon:
            # 행동 중 가장 가능성 있는 행동을 선택
            action = np.argmax(qtable[state,:])
        # 호기심을 억누르지 못하고 모험 시작
        else:
            # 아무데나 일단 가본다
            action = env.action_space.sample()
            
        #행동을 하면 그에 따른 보상과 환경을 받는다.
        new_state, reward, done, _ = env.step(action)

        #지도를 업데이트 한다.
        qtable[state, action] = qtable[state, action] + learning_rate * (reward + gamma * np.max(qtable[new_state, :]) - qtable[state, action])
        
        # 보상을 더한다
        rewards += reward
        
        # 새로운 환경을 적용한다
        state = new_state
        
        # 만약에 죽거나 끝났으면 다른 에피소드로
        if done == True:
            break;
        
    # 지도가 향상되므로 모험의 가능성을 줄인다.
    # 결국 모험의 가능성의 역치는 0.01에 수렴한다.
    epsilon = min_epsilon + (max_epsilon - min_epsilon) * np.exp(-decay_rate * episode)

        

In [8]:
print(qtable)
print(rewards)

[[  0.           0.           0.           0.           0.
    0.        ]
 [ -2.85251635  -2.31564544  -2.85251635  -2.31564544  -1.6445568
  -11.31564544]
 [  0.24288      1.5536       0.24288      1.5536       3.192
   -7.4464    ]
 ...
 [  3.192        5.24         3.192        1.5536      -5.808
   -5.808     ]
 [ -1.6445568   -0.805696    -1.6445568   -0.805696   -10.6445568
  -10.6445568 ]
 [ 11.           7.8         11.          15.           2.
    2.        ]]
-912759


In [9]:
for i in range(100):
    state = env.reset()
    for step in range(50):
        clear_output()
        env.render()
        sleep(0.5)
        action = np.argmax(qtable[state,:])
        
        new_state, reward, done, _ = env.step(action)
        
        state = new_state
        
        if done == True:
            sleep(0.5)
            clear_output()
            env.render()
            sleep(0.5)
            break

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)


KeyboardInterrupt: ignored

In [0]:
print(rewards / total_episodes)